In [1]:

import requests
import chromadb
from base64 import b64encode
from common.payload import data2id
from common.article_scraper import ArticleScraper
from vector_db.utils import ImageEmbeddingFunction, TextEmbeddingFunction, format_document



client = chromadb.HttpClient(host="localhost",port=8000)

#Initialise Image Vector Database
text_fn = TextEmbeddingFunction(remote=True)
text_collection = client.get_or_create_collection(name="text_collection",embedding_function=text_fn)

#Initialise Image Vector Database
img_fn = ImageEmbeddingFunction()
img_collection = client.get_or_create_collection(name="img_collection",embedding_function=img_fn)

headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36"}

#To get query articles
articleScraper = ArticleScraper()

c:\Users\User\anaconda3\envs\web-ext\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
img_url = "https://cdn-attachments.timesofmalta.com/23277ebec8b7be9ab932b5135e534cb4d3dcf80d-1716057657-7e58de73-1920x1280.jpg"
img_data = requests.get(img_url,
                        headers=headers).content
img_bytestring = b64encode(img_data).decode("ascii")


In [23]:
# a = img_collection.query(query_embeddings=img_fn(img_bytestring))
a = img_collection.get(ids="444c8d7b8fb8bc4d2885d57c0c10d057")

In [27]:
print(a)

{'ids': ['444c8d7b8fb8bc4d2885d57c0c10d057'], 'embeddings': None, 'metadatas': [{'article_ids': '["094775e44756453277f15cc095d47712"]', 'captions': '["Joseph Muscat at a news conference shortly after it was confirmed that the magisterial inquiry into the Vitals hospitals deal was concluded. Photo: Matthew Mirabelli"]', 'selectors': '[".ar-ArticleHeader-Standard_sub > picture > img"]'}], 'documents': ['...'], 'data': None, 'uris': None}


In [6]:
query_payload = articleScraper.scrape(
    "https://timesofmalta.com/article/man-dies-traffic-accident-swieqi.1090488"
)
if query_payload.error:
    print(query_payload.error)

query_document = format_document(query_payload.data,query=True)    

x = text_collection.query(
    query_texts=query_document
)
for d,m in zip(x['distances'][0],x['metadatas'][0]):
    print(f"{round(d,2)}\t{m['title']} - {m['url']}")

0.14	Man dies after traffic accident in Swieqi - https://timesofmalta.com/article/man-dies-traffic-accident-swieqi.1090488
0.22	Man dies after Swieqi car crash - https://www.maltatoday.com.mt//news/national/128475/man_dies_after_swieqi_car_crash
0.4	Man injured in Żebbuġ accident - https://timesofmalta.com/article/man-injured-zebbug-accident.1087372
0.42	Man in hospital after traffic accident in Regional Road - https://www.maltatoday.com.mt//news/national/127462/man_in_hospital_after_traffic_accident_in_regional_road
0.44	Man, 35, hit by car in Swieqi - https://timesofmalta.com/article/man-35-hit-car-swieqi.1041104
0.46	Two men injured in separate traffic accidents - https://timesofmalta.com/article/two-men-injured-separate-traffic-accidents.1031498
0.47	Driver seriously injured in three-vehicle crash in St Julian's - https://timesofmalta.com/article/driver-seriously-injured-threevehicle-crash-st-julian.1083528
0.49	Motorcyclist, 73, dies at hospital following Żebbuġ crash - https://ti